# Using OpenCV to Reorder Images based on Point Tags
## Import libraries

In [187]:
import matplotlib as plt
import numpy as np
import cv2 as cv
import os
from pathlib import Path

In [188]:
build_path = Path("Build/")
build_path.mkdir(exist_ok=True)
source_path = Path("Test Data/galaxy/")

#Setup Files for reading
puzzle_pieces = []
with os.scandir(source_path) as entries:
    for entry in entries:
        puzzle_pieces.append(entry.name)

In [189]:
#Store answers
position_answers = []

In [190]:
# Set Marker Color Ranges 
# Note that CV uses BGR instead of RGB
blue_dot = [(200,0,0),(255,40,40),"blue"]
red_dot = [(0,0,200),(40,40,255),"red"]
markers = [blue_dot,red_dot]

In [191]:
def loader(image_filename:str):
    return cv.imread(str(source_path.joinpath(image_filename)))

In [192]:
def solve_position(image_filename:str ) -> dict:
    # test_image = source_path.joinpath(image_filename)
    # loaded_image = cv.imread(str(test_image))
    loaded_image = loader(image_filename)

    #Pre=process image
    blurred_image = cv.medianBlur(loaded_image, 7)

    #Populate total markers in image to Dictionary
    circles = {}
    for lower_bound, upper_bound, color in markers:
        mask = cv.inRange(blurred_image,lower_bound, upper_bound)
        # cv.imshow('mask',mask)
        
        counts = cv.findContours(mask,cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
        circles.update({color : len(counts[0])})
        # cv.destroyWindow('mask')
    
    # cv.destroyAllWindows()
        
    return circles

#Test function works as intended        
assert( solve_position("3Yy2JsTKtsLVG2RU.jpg") == {'blue':2, 'red':2})
    


Columns are Red, Rows are Blue


In [193]:
#go through set of images
for image in puzzle_pieces:
    position = solve_position(image)
    solved_piece = position['red'], position['blue'], image
    position_answers.append(solved_piece)


In [194]:

#find the max row and column
row_size = 0
col_size = 0
for answer in position_answers:
    col_size = max(answer[0],col_size)
    row_size = max(answer[1],row_size)
    
#find shape of total 
single_shape = loader(puzzle_pieces[0]).shape
max_x = col_size * single_shape[0] 
max_y = row_size * single_shape[1] 

In [195]:
row_size, col_size

(30, 30)

In [196]:
max_x, max_y

(6420, 11460)

In [197]:
# Assemble picture

position_answers.sort(key= lambda elem: (elem[0],elem[1]))
horizontal_strip = []
row_section = []

for i in range(row_size):
    row_section.append(i)




count = 0
for i in position_answers:
    print(i[2]," count is at", count)
    horizontal_strip.append(loader(i[2]))
    if (i[1] % col_size ==0):
        print('--------')
        row_section[count] =  np.concatenate(horizontal_strip,axis=0)   #connect single images into horizontal strips
        count += 1
        horizontal_strip.clear()

solution = np.concatenate(row_section, axis=1) #connect horizontal strips of the images      


5JipInjup6cCjPLz.jpg  count is at 0
yjxTRc1khXzNDViN.jpg  count is at 0
kLgrTPWdpvEbVrO4.jpg  count is at 0
XKJ2bIv56OVEVWU4.jpg  count is at 0
u1Ji99ZBbKPYsj6Q.jpg  count is at 0
BQFr5HWum80kM1UF.jpg  count is at 0
HhMpflnCb3hMXR7N.jpg  count is at 0
qx6RV8ukuZSnVf6w.jpg  count is at 0
tcaILVZm0cRATpKU.jpg  count is at 0
hEW58K4O1LP8p6Yg.jpg  count is at 0
YTJ2rWgS5SkB866f.jpg  count is at 0
bbKLcD8T2WzxItOn.jpg  count is at 0
Wh8bVuskMaN1Ysti.jpg  count is at 0
PAeSIkE4xJcYdMGL.jpg  count is at 0
WYLfEA2P6NJxakKX.jpg  count is at 0
g8MNlHktos4zxbDL.jpg  count is at 0
OJk8bjBBD6Pe7Orf.jpg  count is at 0
UD6wZgf8qAhUIQsE.jpg  count is at 0
dhPkXKzSxgivDXXY.jpg  count is at 0
FiZOOEqawZT0n6B6.jpg  count is at 0
yxb7x8u1hEtJjIdj.jpg  count is at 0
OFBPgLbX6IVQFiar.jpg  count is at 0
IjAJ4caZsO0HocsL.jpg  count is at 0
MbtrKbqKj4KM87aU.jpg  count is at 0
jlLVGcmcuhYTEzXB.jpg  count is at 0
A2evXjbsJQ2Ek2Dt.jpg  count is at 0
0StAcBeoy8ApZ0M2.jpg  count is at 0
6Rp0HuW2Yxj5jpYM.jpg  count 

In [198]:
target_filename = 'Completed Puzzle.jpg'
target = str(build_path.joinpath(target_filename))

cv.imwrite(target, solution) 
#Done?

True